# CTEs Introduction

- Common Table Expression
- CTEs allow us to write multi-stage queries in a linear, easy to read format
- Specifies temporary result set(s) that can be referenced by the final query in the CTE
    - Temporary tables that can only be referenced within the full CTE query
- All CTEs begin with a WITH statement, followed with temporary table(s) and a final query that references the temporary tables for a final output

In [1]:
USE AW2019;

WITH
Sales AS (
    SELECT
        OrderDate,
        OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
        TotalDue,
        OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
    FROM Sales.SalesOrderHeader
),

AvgSalesMinusTop10 AS (
    SELECT
        OrderMonth,
        TotalSales  =   SUM (TotalDue)
    FROM Sales
    WHERE OrderRank > 10
    GROUP BY OrderMonth
),

Purchases AS (
    SELECT
        OrderDate,
        OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
        TotalDue,
        OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
    FROM Purchasing.PurchaseOrderHeader
),

AvgPurchasesMinusTop10 AS (
    SELECT
        OrderMonth,
        TotalPurchases = SUM (TotalDue)
    FROM Purchases
    WHERE OrderRank > 10
    GROUP BY OrderMonth
)

SELECT TOP 10
    S.OrderMonth,
    S.TotalSales,
    P.TotalPurchases
FROM AvgSalesMinusTop10 AS S
    INNER JOIN AvgPurchasesMinusTop10 AS P
        ON S.OrderMonth = P.OrderMonth
ORDER BY OrderMonth ASC

(10 rows affected)

Total execution time: 00:00:00.087

OrderMonth,TotalSales,TotalPurchases
2011-12-01,1019635.6747,7254.3006
2012-01-01,3622013.9215,220767.0679
2012-02-01,1141791.6116,7610.834
2012-03-01,2441839.1531,218226.7469
2012-04-01,1341386.2938,2496.2083
2012-05-01,2259194.0397,5744.3167
2012-06-01,3527254.7224,107628.3985
2012-07-01,2560587.9382,234.7418
2012-08-01,1534278.5579,14737.7394
2012-09-01,2903851.4579,3485.3792
